# Подготовка файлов 


In [1]:
!wget https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/gms2.lst
!wget https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/scaffolds.hits_from_SwissProt.txt
!wget https://raw.githubusercontent.com/greggasd/hse21_hw1/main/data/scaffolds.fasta

--2021-12-19 12:22:21--  https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220323 (215K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 215.16K  --.-KB/s    in 0.004s  

2021-12-19 12:22:21 (47.9 MB/s) - ‘gms2.lst’ saved [220323/220323]

--2021-12-19 12:22:21--  https://raw.githubusercontent.com/greggasd/hse21_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1400642 (1.3M) [text/plain]
Saving to: ‘p

In [2]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [4]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 31.6 MB/s 


# Выполнение второго задания

Создаем аннотированный геном бактерии в формате GenBank

Сначала файл *scaffolds*

In [5]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

scaffolds = dict()
for record in SeqIO.parse("scaffolds.fasta", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['organism'] = 'Thalassolituus oleivorans Barents'
  record.annotations['source'] = 'Thalassolituus oleivorans Barents'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'
  
  scaffolds[record.id] = record

Теперь *proteins*

In [6]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1
  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feat)
  genes[desc[0]] = feat

Добавляем функции из бактерии *MIL-1*

In [7]:
mil1genes = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in feat.qualifiers:
    continue
  if 'product' not in feat.qualifiers:
    continue
  
  mil1genes[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

In [9]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
hits = mil1hits[mil1hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1genes[match]]

Финальное для *SwissProt*

In [10]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz
# оставляем только нужное
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > Swiss_names.txt

--2021-12-19 12:38:03--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625798251 (597M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 596.81M   188MB/s    in 3.2s    

2021-12-19 12:38:06 (185 MB/s) - ‘uniprot_sprot.dat.gz’ saved [625798251/625798251]



In [11]:
previd = None
swissgenes = dict()
for line in open('Swiss_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [12]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

Файл GENOME.gbk готов

In [13]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

70